# Topic 3: Recommender System

## Business Objective/Problem
- Tiki là một hệ sinh thái thương mại “all in one”, trong
đó có tiki.vn, là một website thương mại điện tử đứng
top 2 của Việt Nam, top 6 khu vực Đông Nam Á.
- Trên trang này đã triển khai nhiều tiện ích hỗ trợ nâng
cao trải nghiệm người dùng và họ muốn xây dựng
nhiều tiện ích hơn nữa.
- Giả sử công ty này chưa triển khai Recommender
System và bạn được yêu cầu triển khai hệ thống này,
bạn sẽ làm gì?
Đ

## Các kiến thức/ kỹ năng cần để giải quyết
vấn đề này:
- Hiểu vấn đề
- Import các thư viện cần thiết và hiểu cách sử dụng
- Đọc dữ liệu (dữ liệu project này được cung cấp)
- Thực hiện EDA cơ bản (sử dụng Pandas Profiling
Report)
- Tiền xử lý dữ liệu: làm sạch, tạo tính năng mới, lựa
chọn tính năng cần thiết…

## Bước 1: Business Understanding
- Dựa vào yêu cầu nói trên => xác định vấn đề:
  - Chưa có hệ thống Recommendation System
  - => Mục tiêu/ vấn đề: Xây dựng Recommendation System
  cho một hoặc một số nhóm hàng hóa trên tiki.vn giúp đề
  xuất và gợi ý cho người dùng/ khách hàng. => Xây dựng
  các mô hình đề xuất:
    - Content-based filtering
    - Collaborative filtering

# Bước 2: Data Understanding/ Acquire
- Từ mục tiêu/ vấn đề đã xác định: xem xét các dữ
liệu cần thiết:
  - Dữ liệu được cung cấp sẵn gồm có các tập tin:
  ProductRaw.csv, ReviewRaw.csv chứa thông tin sản phẩm,
  review và rating cho các sản phẩm thuộc các nhóm hàng
  hóa như Mobile_Tablet, TV_Audio, Laptop, Camera,
  Accessory

# Bước 3: Data preparation/ Prepare

### 3. Content base Filtering - Cosine Similarity

- Content-based recommendation system based on the same attributes of the product snar to give suggestions to the user

In [1]:
! pip install jieba
! pip install underthesea

In [2]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [3]:
%cd '/content/gdrive/My Drive/LDS0_K271_PhamThiHoa/Project_3'

/content/gdrive/My Drive/LDS0_K271_PhamThiHoa/Project_3


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from underthesea import word_tokenize, pos_tag, sent_tokenize
import warnings
from gensim import corpora, models, similarities
import jieba
import re
import string
import warnings
warnings.filterwarnings("ignore")

In [5]:
# Read data:
products = pd.read_csv('Data/Products.csv')
reviews = pd.read_csv('Data/Reviews.csv',lineterminator='\n')

In [6]:
products.shape

(4373, 10)

In [7]:
products.head(3)

,item_id,name,description,rating,price,list_price,brand,group,url,image
0,48102821,Tai nghe Bluetooth Inpods 12 - Cảm biến vân ta...,THÔNG TIN CHI TIẾT\nDung lượng pin 300\nThời g...,4.0,77000,300000,OEM,Thiết Bị Số - Phụ Kiện Số/Thiết Bị Âm Thanh và...,https://tai-nghe-bluetooth-inpods-12-cam-bien-...,https://salt.tikicdn.com/cache/280x280/ts/prod...
1,52333193,Tai nghe bluetooth không dây F9 True wireless ...,THÔNG TIN CHI TIẾT\nDung lượng pin 2000mah\nTh...,4.5,132000,750000,OEM,Thiết Bị Số - Phụ Kiện Số/Thiết Bị Âm Thanh và...,https://tai-nghe-bluetooth-khong-day-f9-true-w...,https://salt.tikicdn.com/cache/280x280/ts/prod...
2,299461,Chuột Không Dây Logitech M331 Silent - Hàng Ch...,THÔNG TIN CHI TIẾT\nThương hiệu Logitech\nĐộ p...,4.8,299000,399000,Logitech,Thiết Bị Số - Phụ Kiện Số/Phụ kiện máy tính và...,https://chuot-khong-day-logitech-m331-silent-p...,https://salt.tikicdn.com/cache/280x280/media/c...


In [8]:
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4373 entries, 0 to 4372
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   item_id      4373 non-null   int64  
 1   name         4373 non-null   object 
 2   description  4370 non-null   object 
 3   rating       4373 non-null   float64
 4   price        4373 non-null   int64  
 5   list_price   4373 non-null   int64  
 6   brand        4373 non-null   object 
 7   group        4373 non-null   object 
 8   url          4373 non-null   object 
 9   image        4373 non-null   object 
dtypes: float64(1), int64(3), object(6)
memory usage: 341.8+ KB


- We will use columns: 'name' and 'description' to buil model for this topic
- Column name have no Null, column decription has 3 null

In [9]:
# Creat column 'name_descrption' by concatenating column name and description
products['name_description'] = products['name'] + ' ' +  products['description']
products[['name_description']].head()

,name_description
0,Tai nghe Bluetooth Inpods 12 - Cảm biến vân ta...
1,Tai nghe bluetooth không dây F9 True wireless ...
2,Chuột Không Dây Logitech M331 Silent - Hàng Ch...
3,Loa Bluetooth 5.0 Kiêm Đồng Hồ Báo Thức - [[ 2...
4,Tai Nghe Bluetooth Apple AirPods Pro True Wire...


In [10]:
# Check null
products[products['name_description'].isnull()]

,item_id,name,description,rating,price,list_price,brand,group,url,image,name_description
1424,75831435,Camera Ip EZVIZ C8C hồng ngoại không dây Full ...,NaN,0.0,1465000,3200000,EZVIZ,Máy Ảnh - Máy Quay Phim/Camera Giám Sát/Camera IP,https://camera-ip-ezviz-c8c-hong-ngoai-khong-d...,https://salt.tikicdn.com/cache/280x280/ts/prod...,NaN
2504,11607917,Màn Hình LED LG 27MK600M-B 27 inch Full HD (19...,NaN,0.0,4200000,5500000,LG,Laptop - Máy Vi Tính - Linh kiện/Thiết Bị Văn ...,https://man-hinh-led-lg-27mk600m-b-27-inch-ful...,https://salt.tikicdn.com/cache/280x280/ts/prod...,NaN
3263,16608046,Điện Thoại Di Động Forme A5 - Hãng phân phối c...,NaN,3.7,335000,389000,Forme,Điện Thoại - Máy Tính Bảng/Điện thoại phổ thông,https://dien-thoai-di-dong-forme-a5-hang-phan-...,https://salt.tikicdn.com/cache/280x280/ts/prod...,NaN


In [11]:
# Delete rows have null data
products = products[~products['name_description'].isnull()]
# Reset index
products = products.reset_index()

In [12]:
products.shape

(4370, 12)

In [13]:
# Get 5 name_description
for i in range(5):
  print(products.iat[i,11])
  print('-'*20)


Tai nghe Bluetooth Inpods 12 - Cảm biến vân tay, chống nước,màu sắc đa dạng- 5 màu sắc lựa chọn THÔNG TIN CHI TIẾT
Dung lượng pin 300
Thời gian pin - Thời gian nghe nhạc liên tục từ 2.5-4h - Thời gian sạc đầy chỉ khoảng 60p - Thời gian chờ lên tới 140 giờ
Bluetooth 5
Thương hiệu OEM
Xuất xứ thương hiệu Trung Quốc
Độ nhạy cảm biến vân tay
Model i12
Loại Jack cắm USB Cable
Trọng lượng 300g
Thời gian sử dụng
- Thời gian nghe nhạc liên tục từ 2.5-4h
SKU 4096608751631
MÔ TẢ SẢN PHẨM
INPOD 12 là phiên bản nâng cấp mới nhất , tai nghe Bluetooth 5.0 có thiết kế tỉ lệ chuẩn 1:1 với tai Airpod chính hãng
Lược bỏ nút bấm trên thân tai thay vào đó là nút cảm ứng sử dụng dễ dàng và thuận tiện hơn
Thao tác nhận cuộc gọi đến, chuyển nhạc...một cách dễ dàng chỉ bằng 1 chạm
Bluetooth 5.0 mới nhất cho kết nối vô cùng ổn định
- Bluetooth 5.0 mới nhất cho kết nối vô cùng ổn định
- 2 tai nghe tự kết nối với nhau khi được lấy ra từ dock sạc
- Chất âm và thời lượng pin được cải thiện tối ưu
- Dock sạc tiện l

In [14]:
# lower
products['name_description'] = products['name_description'].str.lower()
# Remove sự thông báo
qc = 'giá sản phẩm trên tiki đã bao gồm thuế theo luật hiện hành. tuy nhiên tuỳ vào từng loại sản phẩm hoặc phương thức, địa chỉ giao hàng mà có thể phát sinh thêm chi phí khác như phí vận chuyển, phụ phí hàng cồng kềnh, ...'
products['name_description'] = products['name_description'].str.replace(qc,'')
# Remove some descriptive words
n1 = 'thông tin chi tiết'
n2 = 'mô tả sản phẩm'
n3 = 'tình trạng'
n4 = 'thông số'
n5 = 'hướng dẫn sử dụng'
n6 = 'thương hiệu'
n7 = 'xuất xứ'
for i in [n1,n2,n3,n4,n5,n6,n7]:
  products['name_description'] = products['name_description'].str.replace(i,'')


In [15]:
# Remove Number
products['name_description'] = products['name_description'].str.replace('\d+',' ')
# Remove '\n', '\t' and special character
lst =  ['—','|','\…','•', ',', '.', '\...', '\-', ';', '\?', '\%', '(', ')', '\+', '\/','\"',', ',"\'",'\[','\]','\:','\-','\\n','\\t','\. ','\%','£','$','\"','&','≥','._','±','\*','\//','“','”','._a','v-','v-_a','\–','\\x08','  ','  ','  ','  ']
for i in lst:
  products['name_description'] = products['name_description'].str.replace(i,' ')
# Remove singal character
products['name_description'] = products['name_description'].apply(lambda a: re.sub('\s[a-zA-Z]{1}\s', ' ',a))

In [16]:
# Get 5 name_description
for i in range(5):
  print(products.iat[i,11])
  print('-'*20)

tai nghe bluetooth inpods cảm biến vân tay chống nước màu sắc đa dạng màu sắc lựa chọn dung lượng pin thời gian pin thời gian nghe nhạc liên tục từ thời gian sạc đầy chỉ khoảng thời gian chờ lên tới giờ bluetooth oem trung quốc độ nhạy cảm biến vân tay model loại jack cắm usb cable trọng lượng thời gian sử dụng thời gian nghe nhạc liên tục từ sku inpod là phiên bản nâng cấp mới nhất tai nghe bluetooth có thiết kế tỉ lệ chuẩn với tai airpod chính hãng lược bỏ nút bấm trên thân tai thay vào đó là nút cảm ứng sử dụng dễ dàng và thuận tiện hơn thao tác nhận cuộc gọi đến chuyển nhạc một cách dễ dàng chỉ bằng chạm bluetooth mới nhất cho kết nối vô cùng ổn định bluetooth mới nhất cho kết nối vô cùng ổn định tai nghe tự kết nối với nhau khi được lấy ra từ dock sạc chất âm và thời lượng pin được cải thiện tối ưu dock sạc tiện lợi cho tới lần sạc đầy pin tai nghe các thiết bị tương thích các thiết bị bluetooth như cho xiaomi samsung apple huawei oppo vivo lenovo tablet v thời gian nghe nhạc liên

In [17]:
products['name_description_wt'] = products['name_description'].apply(lambda x: word_tokenize(x, format= 'text'))
products['name_description_wt'].head()

0    tai nghe bluetooth inpods cảm_biến vân tay chố...
1    tai nghe bluetooth không dây true wireless doc...
2    chuột không dây logitech silent hàng chính hãn...
3    loa bluetooth kiêm đồng_hồ báo_thức trong robo...
4    tai nghe bluetooth apple airpods pro true wire...
Name: name_description_wt, dtype: object

In [18]:
# # Postag
# products_postag = products['name_description'].apply(lambda x: pos_tag(x))
# products_postag

In [19]:
# Sent_tokenize(x)
products_sent = products['name_description'].apply(lambda x: sent_tokenize(x))
products_sent

0       [tai nghe bluetooth inpods cảm biến vân tay ch...
1       [tai nghe bluetooth không dây true wireless do...
2       [chuột không dây logitech silent hàng chính hã...
3       [loa bluetooth kiêm đồng hồ báo thức trong rob...
4       [tai nghe bluetooth apple airpods pro true wir...
                              ...                        
4365    [máy lạnh lg inverter hp aph công suất làm lạn...
4366    [micro không dây alpha works chính hãng oem ch...
4367    [máy sấy electrolux kg edv jqsa bảng điều khiể...
4368    [bộ chia pacific dùng chia truyền hình cáp ant...
4369    [micro đôi karaoke micro không dây mv hàng chí...
Name: name_description, Length: 4370, dtype: object

# GEMSIM

In [20]:
# tokenize (split) the sentences into words
products_gem = [[text for text in x.split()] for x in products.name_description_wt]

In [21]:
len(products_gem)

4370

In [22]:
products_gem[:1]

[['tai',
  'nghe',
  'bluetooth',
  'inpods',
  'cảm_biến',
  'vân',
  'tay',
  'chống',
  'nước',
  'màu_sắc',
  'đa_dạng',
  'màu_sắc',
  'lựa_chọn',
  'dung_lượng',
  'pin',
  'thời_gian',
  'pin',
  'thời_gian',
  'nghe',
  'nhạc',
  'liên_tục',
  'từ',
  'thời_gian',
  'sạc',
  'đầy',
  'chỉ',
  'khoảng',
  'thời_gian',
  'chờ',
  'lên',
  'tới',
  'giờ',
  'bluetooth',
  'oem',
  'trung_quốc',
  'độ',
  'nhạy_cảm_biến',
  'vân',
  'tay',
  'model',
  'loại',
  'jack',
  'cắm',
  'usb',
  'cable',
  'trọng_lượng',
  'thời_gian',
  'sử_dụng',
  'thời_gian',
  'nghe',
  'nhạc',
  'liên_tục',
  'từ',
  'sku',
  'inpod',
  'là',
  'phiên_bản',
  'nâng_cấp',
  'mới',
  'nhất',
  'tai_nghe',
  'bluetooth',
  'có',
  'thiết_kế',
  'tỉ_lệ',
  'chuẩn',
  'với',
  'tai',
  'airpod',
  'chính',
  'hãng',
  'lược_bỏ',
  'nút',
  'bấm',
  'trên',
  'thân',
  'tai',
  'thay',
  'vào',
  'đó',
  'là',
  'nút',
  'cảm_ứng',
  'sử_dụng',
  'dễ_dàng',
  'và',
  'thuận_tiện',
  'hơn',
  'thao_tác',


In [23]:
# Load Stopwords
STOP_WORD_FILE = 'Data/vietnamese-stopwords.txt'
with open(STOP_WORD_FILE,'r', encoding='utf-8') as file:
  stop_words = file.read()

stop_words = stop_words.split('\n')
stop_words

['a_lô',
 'a_ha',
 'ai',
 'ai_ai',
 'ai_nấy',
 'ai_đó',
 'alô',
 'amen',
 'anh',
 'anh_ấy',
 'ba',
 'ba_ba',
 'ba_bản',
 'ba_cùng',
 'ba_họ',
 'ba_ngày',
 'ba_ngôi',
 'ba_tăng',
 'bao_giờ',
 'bao_lâu',
 'bao_nhiêu',
 'bao_nả',
 'bay_biến',
 'biết',
 'biết_bao',
 'biết_bao_nhiêu',
 'biết_chắc',
 'biết_chừng_nào',
 'biết_mình',
 'biết_mấy',
 'biết_thế',
 'biết_trước',
 'biết_việc',
 'biết_đâu',
 'biết_đâu_chừng',
 'biết_đâu_đấy',
 'biết_được',
 'buổi',
 'buổi_làm',
 'buổi_mới',
 'buổi_ngày',
 'buổi_sớm',
 'bà',
 'bà_ấy',
 'bài',
 'bài_bác',
 'bài_bỏ',
 'bài_cái',
 'bác',
 'bán',
 'bán_cấp',
 'bán_dạ',
 'bán_thế',
 'bây_bẩy',
 'bây_chừ',
 'bây_giờ',
 'bây_nhiêu',
 'bèn',
 'béng',
 'bên',
 'bên_bị',
 'bên_có',
 'bên_cạnh',
 'bông',
 'bước',
 'bước_khỏi',
 'bước_tới',
 'bước_đi',
 'bạn',
 'bản',
 'bản_bộ',
 'bản_riêng',
 'bản_thân',
 'bản_ý',
 'bất_chợt',
 'bất_cứ',
 'bất_giác',
 'bất_kì',
 'bất_kể',
 'bất_kỳ',
 'bất_luận',
 'bất_ngờ',
 'bất_nhược',
 'bất_quá',
 'bất_quá_chỉ',
 'bất_thình_l

In [24]:
# Remove stopword
products_gem_re = [[t for t in text if not t in stop_words] for text in products_gem] 
# Remove alphabet again
# products_gem_re = [[t for t in text if not t in list(string.ascii_lowercase)] for text in products_gem_re] 

In [25]:
products_gem_re[:1]

[['tai',
  'bluetooth',
  'inpods',
  'cảm_biến',
  'vân',
  'chống',
  'màu_sắc',
  'đa_dạng',
  'màu_sắc',
  'lựa_chọn',
  'dung_lượng',
  'pin',
  'pin',
  'nhạc',
  'liên_tục',
  'sạc',
  'chờ',
  'bluetooth',
  'oem',
  'trung_quốc',
  'độ',
  'nhạy_cảm_biến',
  'vân',
  'model',
  'jack',
  'cắm',
  'usb',
  'cable',
  'trọng_lượng',
  'nhạc',
  'liên_tục',
  'sku',
  'inpod',
  'phiên_bản',
  'nâng_cấp',
  'tai_nghe',
  'bluetooth',
  'thiết_kế',
  'tỉ_lệ',
  'chuẩn',
  'tai',
  'airpod',
  'hãng',
  'lược_bỏ',
  'nút',
  'bấm',
  'thân',
  'tai',
  'thay',
  'nút',
  'cảm_ứng',
  'dễ_dàng',
  'thuận_tiện',
  'thao_tác',
  'nhạc',
  'dễ_dàng',
  'chạm',
  'bluetooth',
  'kết_nối',
  'vô_cùng',
  'ổn_định',
  'bluetooth',
  'kết_nối',
  'vô_cùng',
  'ổn_định',
  'tai_nghe',
  'kết_nối',
  'dock',
  'sạc',
  'chất',
  'âm',
  'thời_lượng',
  'pin',
  'cải_thiện',
  'tối_ưu',
  'dock',
  'sạc',
  'tiện_lợi',
  'sạc',
  'pin',
  'tai_nghe',
  'thiết_bị',
  'tương_thích',
  'thiết_bị

In [26]:
# Obitain the number of fuatures based on dictionary: Use corpora. Dictionary
dictionary = corpora.Dictionary(products_gem_re)

In [27]:
# List of features in dictionary
dictionary.token2id

{'airpod': 0,
 'apple': 1,
 'bluetooth': 2,
 'bấm': 3,
 'cable': 4,
 'chuẩn': 5,
 'chạm': 6,
 'chất': 7,
 'chống': 8,
 'chờ': 9,
 'cải_thiện': 10,
 'cảm_biến': 11,
 'cảm_ứng': 12,
 'cắm': 13,
 'dock': 14,
 'dung_lượng': 15,
 'dễ_dàng': 16,
 'huawei': 17,
 'hãng': 18,
 'inpod': 19,
 'inpods': 20,
 'jack': 21,
 'kết_nối': 22,
 'lenovo': 23,
 'liên_tục': 24,
 'lược_bỏ': 25,
 'lựa_chọn': 26,
 'model': 27,
 'màu_sắc': 28,
 'nhạc': 29,
 'nhạy_cảm_biến': 30,
 'nâng_cấp': 31,
 'nút': 32,
 'oem': 33,
 'oppo': 34,
 'phiên_bản': 35,
 'pin': 36,
 'samsung': 37,
 'sku': 38,
 'sạc': 39,
 'tablet': 40,
 'tai': 41,
 'tai_nghe': 42,
 'thao_tác': 43,
 'thay': 44,
 'thiết_bị': 45,
 'thiết_kế': 46,
 'thuận_tiện': 47,
 'thân': 48,
 'thời_lượng': 49,
 'tiện_lợi': 50,
 'trung_quốc': 51,
 'trọng_lượng': 52,
 'tương_thích': 53,
 'tỉ_lệ': 54,
 'tối_ưu': 55,
 'usb': 56,
 'vivo': 57,
 'vân': 58,
 'vô_cùng': 59,
 'xiaomi': 60,
 'âm': 61,
 'đa_dạng': 62,
 'độ': 63,
 'ổn_định': 64,
 'android': 65,
 'báo': 66,
 'bật'

In [28]:
len(dictionary.token2id)

16903

In [29]:
# Numbers of features (word) in dictionary
feature_cnt = len(dictionary.token2id)
feature_cnt

16903

In [30]:
# Obtain corpus based on dictionary (dense matrix)
corpus = [dictionary.doc2bow(text) for text in products_gem_re]

In [31]:
corpus[0]

[(0, 1),
 (1, 1),
 (2, 6),
 (3, 1),
 (4, 1),
 (5, 1),
 (6, 1),
 (7, 1),
 (8, 1),
 (9, 2),
 (10, 1),
 (11, 1),
 (12, 1),
 (13, 1),
 (14, 2),
 (15, 1),
 (16, 2),
 (17, 1),
 (18, 1),
 (19, 1),
 (20, 1),
 (21, 1),
 (22, 3),
 (23, 1),
 (24, 3),
 (25, 1),
 (26, 1),
 (27, 1),
 (28, 2),
 (29, 4),
 (30, 1),
 (31, 1),
 (32, 2),
 (33, 1),
 (34, 1),
 (35, 1),
 (36, 4),
 (37, 1),
 (38, 1),
 (39, 5),
 (40, 1),
 (41, 3),
 (42, 3),
 (43, 1),
 (44, 1),
 (45, 2),
 (46, 1),
 (47, 1),
 (48, 1),
 (49, 1),
 (50, 1),
 (51, 1),
 (52, 1),
 (53, 1),
 (54, 1),
 (55, 1),
 (56, 1),
 (57, 1),
 (58, 2),
 (59, 2),
 (60, 1),
 (61, 1),
 (62, 1),
 (63, 1),
 (64, 2)]

In [32]:
# Use TF_IDF Model to process corpus, obtaining index
tfidf = models.TfidfModel(corpus)
# Tính toán sự tương tự trong ma trận thua thớt
index = similarities.SparseMatrixSimilarity(tfidf[corpus], num_features= feature_cnt)

In [33]:
# Đề xuất cho sản phẩm đang xem:
def recomender(item_ID, dictionary, tfidf, index):
  product_selection = products[products['item_id'] == item_ID]
  name_description_pre = product_selection['name_description_wt'].to_string(index = False)

  # Convert search words into sparse Vectors
  view_product = name_description_pre.lower().split()
  kw_vector = dictionary.doc2bow(view_product)

  # Similary caculation
  sim = index[tfidf[kw_vector]]
  # print result
  list_id =[] 
  list_score = []
  for i in range(len(sim)):
    list_id.append(i)
    list_score.append(sim[i])
  df_result = pd.DataFrame({'id': list_id, 'score': list_score})
  # Find high score:
  five_highest_score = df_result.sort_values(by = 'score', ascending = False).head(6)
  idToList = list(five_highest_score['id'])
  # print(idToList)

  product_find = products[products['index'].isin(idToList)]
  # print(product_find)
  results = product_find[['index','item_id','name']]
  # results = pd.concat([results, five_highest_score], axis = 1).sort_values(by='score', ascending = False)
  results = results.merge(five_highest_score, left_on= 'index', right_on='id').sort_values(by='score', ascending = False)
  name = results[results['item_id']== item_ID].reset_index()['name'].to_string(index = False)
  results = results[results['item_id'] != item_ID].reset_index()
  return results , name

In [34]:
# Đề xuất cho sản phẩm đang xem với item_id = 48102821
item_ID = 48102821
results, name  = recomender(item_ID, dictionary, tfidf, index)
print(f'Recomend for product Id: {item_ID}\nName product: {name}')
results

Recomend for product Id: 48102821
Name product:  Tai nghe Bluetooth Inpods 12 - Cảm biến vân ta...


,level_0,index,item_id,name,id,score
0,1,75,35607267,Tai nghe Bluetooth Inpods 12 Thời trang,75,0.273726
1,4,299,74247140,Tai nghe không dây Bluetooth Inpod 12 Simple -...,299,0.214945
2,2,77,70772235,Tai nghe bluetooth nhét tai không dây tai phon...,77,0.214697
3,3,216,54406343,Tai Nghe Bluetooth Nhét Tai Không Dây 5.0 Dalu...,216,0.187997
4,5,689,67634480,Tai Nghe Bluetooth True Wireless AMOI F9 5.0 C...,689,0.164717


In [35]:
# Đề xuất cho sản phẩm đang xem với item_id = 299461
item_ID = 299461
results, name  = recomender(item_ID, dictionary, tfidf, index)
print(f'Recomend for product Id: {item_ID}\nName: {name}')
results

Recomend for product Id: 299461
Name:  Chuột Không Dây Logitech M331 Silent - Hàng Ch...


,level_0,index,item_id,name,id,score
0,1,28,916784,Chuột Không Dây Logitech M590 Slient - Hàng Ch...,28,0.515210
1,2,102,67907870,Chuột Bluetooth Silent Logitech Pebble M350 - ...,102,0.505742
2,5,893,76781356,Chuột không dây LOGITECH M331 Black Cổng USB -...,893,0.499403
3,3,141,4312979,Chuột Không Dây Logitech M220 - Hàng Chính Hãng,141,0.457892
4,4,355,515550,Bộ Bàn Phím Và Chuột Không Dây Logitech MK235 ...,355,0.457264


In [36]:
# Đề xuất cho sản phẩm đang xem với item_id = 34790708
item_ID = 34790708
results, name  = recomender(item_ID, dictionary, tfidf, index)
print(f'Recomend for product Id: {item_ID}\nName: {name}')
results

Recomend for product Id: 34790708
Name:  SIM 4G I-Telecom gói cước MAY tặng 90GB DATA, ...


,level_0,index,item_id,name,id,score
0,1,265,51030375,"Sim 4G Viettel V120 Tặng 2GB/Ngày, Miễn Phí Gọ...",265,0.434368
1,4,863,4216333,Sim 4G Mobifone C90N Tặng 4GB / Ngày Và 1000 P...,863,0.428682
2,0,40,52087768,Sim 4G MAY V77 (Vinaphone-Itelecom) gói cước 9...,40,0.424293
3,5,898,51343843,"Sim 4G Vinaphone D60G Tặng 2GB/Ngày, Miễn Phí ...",898,0.423251
4,3,720,56873256,Sim 4G Mobifone C120N 120GB (4GB/ngày) gọi nội...,720,0.406156


# Save Results

In [37]:
import pickle

In [39]:
dictionary.save('Results/Gensim/dictionary')
corpora.MmCorpus.serialize('Results/Gensim/corpus', corpus)

In [40]:
with open('Results/Gensim/tfidf.pk', 'wb') as fin:
  pickle.dump(tfidf, fin)
with open('Results/Gensim/index.pk', 'wb') as file:
  pickle.dump(index,file)

In [41]:
# Load
dic = corpora.Dictionary.load('Results/Gensim/dictionary')
corp = corpora.MmCorpus('Results/Gensim/corpus')

In [42]:
dic.token2id

{'airpod': 0,
 'apple': 1,
 'bluetooth': 2,
 'bấm': 3,
 'cable': 4,
 'chuẩn': 5,
 'chạm': 6,
 'chất': 7,
 'chống': 8,
 'chờ': 9,
 'cải_thiện': 10,
 'cảm_biến': 11,
 'cảm_ứng': 12,
 'cắm': 13,
 'dock': 14,
 'dung_lượng': 15,
 'dễ_dàng': 16,
 'huawei': 17,
 'hãng': 18,
 'inpod': 19,
 'inpods': 20,
 'jack': 21,
 'kết_nối': 22,
 'lenovo': 23,
 'liên_tục': 24,
 'lược_bỏ': 25,
 'lựa_chọn': 26,
 'model': 27,
 'màu_sắc': 28,
 'nhạc': 29,
 'nhạy_cảm_biến': 30,
 'nâng_cấp': 31,
 'nút': 32,
 'oem': 33,
 'oppo': 34,
 'phiên_bản': 35,
 'pin': 36,
 'samsung': 37,
 'sku': 38,
 'sạc': 39,
 'tablet': 40,
 'tai': 41,
 'tai_nghe': 42,
 'thao_tác': 43,
 'thay': 44,
 'thiết_bị': 45,
 'thiết_kế': 46,
 'thuận_tiện': 47,
 'thân': 48,
 'thời_lượng': 49,
 'tiện_lợi': 50,
 'trung_quốc': 51,
 'trọng_lượng': 52,
 'tương_thích': 53,
 'tỉ_lệ': 54,
 'tối_ưu': 55,
 'usb': 56,
 'vivo': 57,
 'vân': 58,
 'vô_cùng': 59,
 'xiaomi': 60,
 'âm': 61,
 'đa_dạng': 62,
 'độ': 63,
 'ổn_định': 64,
 'android': 65,
 'báo': 66,
 'bật'

In [43]:
corp[0]

[(0, 1.0),
 (1, 1.0),
 (2, 6.0),
 (3, 1.0),
 (4, 1.0),
 (5, 1.0),
 (6, 1.0),
 (7, 1.0),
 (8, 1.0),
 (9, 2.0),
 (10, 1.0),
 (11, 1.0),
 (12, 1.0),
 (13, 1.0),
 (14, 2.0),
 (15, 1.0),
 (16, 2.0),
 (17, 1.0),
 (18, 1.0),
 (19, 1.0),
 (20, 1.0),
 (21, 1.0),
 (22, 3.0),
 (23, 1.0),
 (24, 3.0),
 (25, 1.0),
 (26, 1.0),
 (27, 1.0),
 (28, 2.0),
 (29, 4.0),
 (30, 1.0),
 (31, 1.0),
 (32, 2.0),
 (33, 1.0),
 (34, 1.0),
 (35, 1.0),
 (36, 4.0),
 (37, 1.0),
 (38, 1.0),
 (39, 5.0),
 (40, 1.0),
 (41, 3.0),
 (42, 3.0),
 (43, 1.0),
 (44, 1.0),
 (45, 2.0),
 (46, 1.0),
 (47, 1.0),
 (48, 1.0),
 (49, 1.0),
 (50, 1.0),
 (51, 1.0),
 (52, 1.0),
 (53, 1.0),
 (54, 1.0),
 (55, 1.0),
 (56, 1.0),
 (57, 1.0),
 (58, 2.0),
 (59, 2.0),
 (60, 1.0),
 (61, 1.0),
 (62, 1.0),
 (63, 1.0),
 (64, 2.0)]